In [1]:
!pip install pymorphy3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 58.6 MB/s eta 0:00:00


In [91]:
import re
import sqlite3
from pymorphy3 import MorphAnalyzer

In [92]:
morph = MorphAnalyzer(lang='ru')

In [93]:
pos_tags = ('NOUN', 'ADJ', 'VERB', 'ADV', 'PROPN', 'INTJ', 'PRON', 'DET', 'NUM', 'CCONJ', 'SCONJ', 'ADP', 'PART', 'AUX')
deprel_tags = ('acl', 'acl:relcl', 'advcl', 'advmod', 'amod', 'appos', 'aux',
               'aux:pass', 'case', 'cc', 'ccomp', 'conj', 'cop', 'csubj', 'det',
               'discourse', 'expl', 'fixed', 'flat', 'flat:name', 'iobj',
               'list', 'mark', 'nmod', 'nsubj', 'nsubj:pass', 'nummod',
               'nummod:gov', 'obj', 'obl', 'obl:agent', 'parataxis', 'root',
               'vocative', 'xcomp')

In [94]:
def parse_single_token(token):
  """Эта функция принимает однокомпонентный (без '+') токен
        (слово в кавычках/слово без кавычек/POS-тег/deprel-тег) и выдаёт
        кортеж, где первый элемент — "режим поиска", а второй — элемент, по
        которому будет осуществляться поиск."""
  if token in pos_tags:  # Поиск по части речи
    return 'only_pos', token
  elif token in deprel_tags:  # Поиск по синтаксической функции
    return 'only_deprel', token
  else:
    if all(char.isascii() for char in token.strip('"').lower()): # Проверяем,
    # что введённое пользователем слово — не полностью написанное латиницей (при
    # этом часть символов могут быть латинскими, например "it-специалиста" —
    # валидный запрос)
      print('Слово не может быть полностью написано латиницей! Если же вы имели в виду тег, то среди доступных его нет, проверьте инструкцию по поиску!')
      return None
    elif re.match(r'^"(.+)"$', token):  # Поиск точной формы
      return 'exact_wordform', token[1:-1].lower()
    else:  # Поиск любых грамматических форм
      lemma = morph.normal_forms(token)[0]
      return 'lemma', lemma

In [95]:
def parse_multipart_token(token):
    """Эта функция принимает составной (с '+') токен (лемму с тегами — POS
        и/или deprel), разбивает по '+' и выдаёт кортеж, где первый элемент —
        "режим поиска", а последующие — элементы, по которым будет
        осуществляться поиск."""
    parts = token.split('+')
    if len(parts) > 3:
      print('В токене может быть макс. 3 элемента — лемма, POS-тег и deprel-тег!')
      return None
    else:
      lemma = parts[0].lower()
      if len(parts) == 2:
        if parts[1] in pos_tags:
          return 'lemma+pos', lemma, parts[1]
        elif parts[1] in deprel_tags:
          return 'lemma+deprel', lemma, parts[1]
        else:
          print('Введённого тега среди доступных нет, проверьте инструкцию по поиску!')
          return None
      else:
        if parts[1] in pos_tags and parts[2] in deprel_tags:
          return 'lemma+pos+deprel', lemma, parts[1], parts[2]
        else:
          if parts[1] not in pos_tags and parts[2] in deprel_tags:
            print('Введённого POS-тега среди доступных нет, проверьте инструкцию по поиску!')
            return None
          elif parts[1] in pos_tags and parts[2] not in deprel_tags:
            print('Введённого deprel-тега среди доступных нет, проверьте инструкцию по поиску!')
            return None
          elif parts[1] in deprel_tags and parts[2] in pos_tags:
            print('Неверный порядок ввода тегов: необходимо ввести сперва POS-, а затем — deprel-тег!')
            return None
          else:
            print('Введённых тегов среди доступных нет, проверьте инструкцию по поиску!')
            return None

In [96]:
def process_query(user_request):
  """Эта функция обрабатывает пользовательский запрос и выдаёт список
        кортежей для дальнейшего составления запросов к базе данных."""
  if re.search(r'[^a-zA-Zа-яА-ЯёЁ +:"]', user_request):
    value_error = 'В запросе присутствуют некорректные символы!'
    return value_error
  else:
    tokens = user_request.strip().split()
    if len(tokens) > 3:
      value_error = 'Длина запроса превышает 3 токена!'
      return value_error
    else:
      parsed_tokens_for_search = []
      for token in tokens:
        if '+' in token:
          result = parse_multipart_token(token)
          if isinstance(result, tuple):
            parsed_tokens_for_search.append(result)
          else:
            value_error = result
            return value_error
        else:
          result = parse_single_token(token)
          if isinstance(result, tuple):
            parsed_tokens_for_search.append(result)
          else:
            value_error = result
            return value_error
      return parsed_tokens_for_search

In [97]:
correct_user_request = 'компьютер+NOUN+nsubj'
correct_user_request_list = process_query(correct_user_request)
correct_user_request_list

[('lemma+pos+deprel', 'компьютер', 'NOUN', 'nsubj')]

In [98]:
incorrect_user_request = 'компьютер+nsubj+NOUN'
incorrect_user_request_list = process_query('компьютер+nsubj+NOUN')

Неверный порядок ввода тегов: необходимо ввести сперва POS-, а затем — deprel-тег!


In [99]:
print(incorrect_user_request_list)

None


In [100]:
multiple_tokens_user_request = 'компьютер+NOUN+nsubj VERB'
multiple_tokens_user_request_list = process_query('компьютер+NOUN+nsubj VERB')
multiple_tokens_user_request_list

[('lemma+pos+deprel', 'компьютер', 'NOUN', 'nsubj'), ('only_pos', 'VERB')]

In [41]:
conn = sqlite3.connect('habr_corpus.db')
cur = conn.cursor()

In [42]:
def search_exact_wordform(token_info_tuple):
    """Эта функция принимает кортеж с информацией о токене, составляет
        SQL-запрос для поиска по конкретной словоформе и выдаёт список
        кортежей из id слов и id предложений из базы данных, подходящих под
        запрос."""
    query_condition = """
    SELECT word_id, sent_id
    FROM words
    WHERE token = ?
    """
    cur.execute(query_condition, (token_info_tuple[1],))
    output = cur.fetchall()
    return output

In [43]:
def search_lemma(token_info_tuple):
    """Эта функция принимает кортеж с информацией о токене, составляет
        SQL-запрос для поиска любых грамматических форм и выдаёт список
        кортежей из id слов и id предложений из базы данных, подходящих под
        запрос."""
    query_condition = """
    SELECT word_id, sent_id
    FROM words
    WHERE lemma = ?
    """
    cur.execute(query_condition, (token_info_tuple[1],))
    output = cur.fetchall()
    return output

In [44]:
def search_only_pos(token_info_tuple):
    """Эта функция принимает кортеж с информацией о токене, составляет
        SQL-запрос для поиска по части речи и выдаёт список кортежей из id
        слов и id предложений из базы данных, подходящих под запрос."""
    query_condition = """
    SELECT word_id, sent_id
    FROM words
    WHERE pos = ?
    """
    cur.execute(query_condition, (token_info_tuple[1],))
    output = cur.fetchall()
    return output

In [45]:
def search_only_deprel(token_info_tuple):
    """Эта функция принимает кортеж с информацией о токене, составляет
        SQL-запрос для поиска по синтаксической функции и выдаёт список
        кортежей из id слов и id предложений из базы данных, подходящих под
        запрос."""
    query_condition = """
    SELECT word_id, sent_id
    FROM words
    WHERE deprel = ?
    """
    cur.execute(query_condition, (token_info_tuple[1],))
    output = cur.fetchall()
    return output

In [46]:
def search_lemma_and_pos(token_info_tuple):
    """Эта функция принимает кортеж с информацией о токене, составляет
        SQL-запрос для поиска по лемме со специфицированной частью речи и
        выдаёт список кортежей из id слов и id предложений из базы данных,
        подходящих под запрос."""
    query_condition = """
    SELECT word_id, sent_id
    FROM words
    WHERE lemma = ? AND pos = ?
    """
    cur.execute(query_condition, (token_info_tuple[1], token_info_tuple[2]))
    output = cur.fetchall()
    return output

In [47]:
def search_lemma_and_deprel(token_info_tuple):
    """Эта функция принимает кортеж с информацией о токене, составляет
        SQL-запрос для поиска по лемме со специфицированной синтаксической
        функцией и выдаёт список кортежей из id слов и id предложений из базы
        данных, подходящих под запрос."""
    query_condition = """
    SELECT word_id, sent_id
    FROM words
    WHERE lemma = ? AND deprel = ?
    """
    cur.execute(query_condition, (token_info_tuple[1], token_info_tuple[2]))
    output = cur.fetchall()
    return output

In [48]:
def search_lemma_and_pos_and_deprel(token_info_tuple):
    """Эта функция принимает кортеж с информацией о токене, составляет
        SQL-запрос для поиска по лемме со специфицированными частью речи И
        синтаксической функцией и выдаёт список кортежей из id слов и id
        предложений из базы данных, подходящих под запрос."""
    query_condition = """
    SELECT word_id, sent_id
    FROM words
    WHERE lemma = ? AND pos = ? AND deprel = ?
    """
    cur.execute(query_condition, (token_info_tuple[1], token_info_tuple[2], token_info_tuple[3]))
    output = cur.fetchall()
    return output

In [77]:
output_example = search_lemma_and_pos_and_deprel(('lemma+pos+deprel', 'компьютер', 'NOUN', 'nsubj'))
output_example

[(35488, 2506),
 (79850, 5899),
 (100765, 7655),
 (161361, 12169),
 (169720, 12722),
 (179111, 13410),
 (212291, 15898)]

In [81]:
def get_output_for_next_token(output, next_token_info_tuple):
    """Эта функция принимает список кортежей из id слов и id предложений,
        в которых содержится первый токен, из базы данных, а также кортеж с
        информацией о следующем токене и выдаёт список кортежей из id слов и
        id предложений, в которых содержатся первый и следующий токены, из
        базы данных."""
    next_token_output = []
    if next_token_info_tuple[0] == 'exact_wordform':
        element = 'token'
    elif next_token_info_tuple[0] == 'only_pos':
        element = 'pos'
    elif next_token_info_tuple[0] == 'only_deprel':
        element = 'deprel'
    else:
        element = 'lemma'
        if next_token_info_tuple[0] == 'lemma+pos':
            element += ' = ? AND pos'
        elif next_token_info_tuple[0] == 'lemma+deprel':
            element += ' = ? AND deprel'
        elif next_token_info_tuple[0] == 'lemma+pos+deprel':
            element += ' = ? AND pos = ? AND deprel'
    next_token_query_condition = f"""
    SELECT word_id, sent_id
    FROM words
    WHERE {element} = ? AND word_id = ? AND sent_id = ?
    """
    for word_id, sent_id in output:
        next_word_id = word_id + 1
        if ' ' in element:
            elements = element.split(' = ? AND ')
            if len(elements) == 2:
                cur.execute(next_token_query_condition, (next_token_info_tuple[1], next_token_info_tuple[2], next_word_id, sent_id))
            else:
                cur.execute(next_token_query_condition, (next_token_info_tuple[1], next_token_info_tuple[2], next_token_info_tuple[3], next_word_id, sent_id))
        else:
            cur.execute(next_token_query_condition, (next_token_info_tuple[1], next_word_id, sent_id))
        result = cur.fetchone()
        if result:
            next_token_output.append(result)
    return next_token_output

In [82]:
get_output_for_next_token(output_example, ('only_pos', 'VERB'))

[(79851, 5899)]

In [83]:
def get_sentences_idxs(parsed_tokens_for_search):
    """Эта функция принимает список кортежей с разбором 1-3 токенов, с
        помощью функций выше преобразует в SQL-запросы к базе данных и выдаёт
        список id предложений, подходящих под пользовательский запрос."""
    if parsed_tokens_for_search[0][0] == 'exact_wordform':
        output = search_exact_wordform(parsed_tokens_for_search[0])
    elif parsed_tokens_for_search[0][0] == 'lemma':
        output = search_lemma(parsed_tokens_for_search[0])
    elif parsed_tokens_for_search[0][0] == 'only_pos':
        output = search_only_pos(parsed_tokens_for_search[0])
    elif parsed_tokens_for_search[0][0] == 'only_deprel':
        output = search_only_deprel(parsed_tokens_for_search[0])
    elif parsed_tokens_for_search[0][0] == 'lemma+pos':
        output = search_lemma_and_pos(parsed_tokens_for_search[0])
    elif parsed_tokens_for_search[0][0] == 'lemma+deprel':
        output = search_lemma_and_deprel(parsed_tokens_for_search[0])
    else:
        output = search_lemma_and_pos_and_deprel(parsed_tokens_for_search[0])

    if output and len(parsed_tokens_for_search) > 1:
        next_token_output = get_output_for_next_token(output, parsed_tokens_for_search[1])
        if len(parsed_tokens_for_search) == 3:
            next_token_output = get_output_for_next_token(next_token_output, parsed_tokens_for_search[2])
            sentences_idxs = [result[1] for result in next_token_output]
            return sentences_idxs
        else:
            sentences_idxs = [result[1] for result in next_token_output]
            return sentences_idxs  # получаем список айдишников предложений
    if output:
        output = [result[1] for result in output]
    return output

In [84]:
print(get_sentences_idxs(correct_user_request_list))

[2506, 5899, 7655, 12169, 12722, 13410, 15898]


In [85]:
idxs = get_sentences_idxs(correct_user_request_list)

In [86]:
multiple_tokens_user_request_idxs = get_sentences_idxs(multiple_tokens_user_request_list)

In [87]:
def get_sentences_text_with_metainfo(idxs):
    """Эта функция принимает id предложений, подходящих под
        пользовательский запрос, составляет SQL-запрос с использованием JOIN
        для объединения данных из таблиц sentences и articles базы данных и
        выдаёт предложения с метаинформацией (автор, заголовок, ссылка на
        статью)."""
    if len(idxs) == 1:
        idxs = f'({idxs[0]})'
    else:
        idxs = tuple(idxs)
    sentences_with_metainfo_query_condition = f'''
        SELECT DISTINCT sentence, author, title, link FROM sentences
        JOIN articles ON sentences.article_id=articles.article_id
        WHERE sentences.sent_id in {idxs}
    '''
    cur = conn.cursor()
    cur.execute(sentences_with_metainfo_query_condition)
    sentences_with_metainfo = cur.fetchall()
    return sentences_with_metainfo

In [88]:
get_sentences_text_with_metainfo(idxs)

[('Представьте себе 1970-е годы, время, когда компьютеры только начинали проникать в повседневную жизнь.',
  'gnatyuk_sergey',
  'Objective-C не кусается: как перестать бояться Legacy и стать настоящим iOS-ниндзя / Хабр',
  'https://habr.com//ru/articles/848788/'),
 ('С такими людьми компьютеры становятся вдвойне веселее.',
  'spring_aio',
  'Взлом JVM-приложения с помощью отладчика IntelliJ IDEA / Хабр',
  'https://habr.com//ru/companies/spring_aio/articles/845554/'),
 ('Объективно, не у всех ребят очень мощные компьютеры.',
  'Aeliot',
  'Автоматизация CQC на CI / Хабр',
  'https://habr.com//ru/articles/852978/'),
 ('И хотя для практического применения Q-Newton потребуются достаточно мощные квантовые компьютеры, подобные гибридные схемы имеют все шансы преодолеть вычислительные ограничения классических систем.',
  'breakmirrors',
  '«А можно быстрее?»: практические советы по ускорению обучения нейросетей / Хабр',
  'https://habr.com//ru/companies/magnus-tech/articles/846012/'),
 ('Жа

In [89]:
get_sentences_text_with_metainfo(multiple_tokens_user_request_idxs)

[('С такими людьми компьютеры становятся вдвойне веселее.',
  'spring_aio',
  'Взлом JVM-приложения с помощью отладчика IntelliJ IDEA / Хабр',
  'https://habr.com//ru/companies/spring_aio/articles/845554/')]